### Hi everyone, I'm Abrar Jahin Niloy from Team Cristiano_Ronaldo. Here's merged My Training and Inference Notebook.

#### What I actually did:
1. Initially I tried different types of model and got a good score in Byt5 transformer. So I continued with this model.
2. First I Pretrained "Byt5-small" model on a simillar type competition 'DataVerse Challenge - ITVerse 2023'  
    I Pre-trained the model according to first prize solution notebook 
    Reference Notebook: [First Prize solution training notebook](https://www.kaggle.com/code/umongsain/training-notebook)   
    My Pre-Train Notebook: [Pre-Trained on Dataverse](https://www.kaggle.com/code/abrarjahinniloy/byte-t5-training-on-dataverse/notebook?scriptVersionId=163318207)  
    Basically Both notebooks are same. I just copied from the reference notebook.  
    Pre-Trained Checkpoint Link: [Model Checkpoint Trained on Dataverse](https://www.kaggle.com/datasets/abrarjahinniloy/model-checkpoint-trained-on-dataverse/data)  
3.Then I Fine tunned with this checkpoint on this Bhashamul competition dataset.  
  Final Training Notebook Link: [Training Notebook See Version 2](https://www.kaggle.com/code/abrarjahinniloy/bhashamul-ipa-final-training/notebook)  
  Final Infererce Notebook Link:[Inference Notebook See Version 19](https://www.kaggle.com/code/abrarjahinniloy/bhashamul-bengali-regional-ipa-inference-2/notebook)   
4. I have mentioned my detailed workflow on video presentation.



## EDA (Exploratory Data Analysis)

In [ ]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
train_df = pd.read_csv("/kaggle/input/regipa/train_regipa.csv")
test_df = pd.read_csv("/kaggle/input/regipa/test_regipa.csv", index_col="Index")

In [ ]:
train_df = train_df.drop(columns=['Index','District'])
train_df.head()

In [ ]:
test_df = test_df.drop(columns=['District'])
test_df.head()

In [ ]:
display(train_df.isna().value_counts())
display(test_df.isna().value_counts())

In [ ]:
train_df.dropna(inplace=True)
test_df.dropna(inplace=True)

In [ ]:
print("Total number of training samples:", len(train_df))
print("Total number of test samples:", len(test_df))

In [ ]:
fig, axes = plt.subplots(1, 2, figsize=(15, 5), sharey=True)

train_lengths = train_df["Contents"].str.len()
sns.histplot(ax=axes[0], data=train_lengths, bins=10).set(xlabel="Length of training text samples")
axes[0].axvline(train_lengths.mean(), c="k", ls="--", lw=2.5, label="Mean")
axes[0].legend()

test_lengths = test_df["Contents"].str.len()
sns.histplot(ax=axes[1], data=test_lengths, bins=10).set(xlabel="Length of test text samples")
axes[1].axvline(test_lengths.mean(), c="k", ls="--", lw=2.5, label="Mean")
axes[1].legend()


plt.show()

In [ ]:
# For calculating WER (Word Error Rate)
! pip install jiwer

## Training

**I used a 10% validation dataset to train the model and at 17000 steps, I achieved a respectable score. So now I will train using nearly the entire training data set while maintaining the same model parameters. **

In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split

train_df, val_df = train_test_split(train_df, test_size=0.05, shuffle=True, random_state=3000)
train_df = train_df.reset_index(drop=True)
val_df = val_df.reset_index(drop=True)

In [ ]:
from datasets import Dataset

ds_train = Dataset.from_pandas(train_df)
ds_eval = Dataset.from_pandas(val_df)

## Model

**For this dataset, I primarily train with 'google/byt5-small' Transformer. First, I train the model using the 'DataVerse Challenge - ITVerse 2023' competition dataset. Use the model checkpoint to train on this dataset. **

In [ ]:
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM, DataCollatorForSeq2Seq

model_id = "/kaggle/input/model-checkpoint-trained-on-dataverse/Model/"
tokenizer = AutoTokenizer.from_pretrained(model_id)
model = AutoModelForSeq2SeqLM.from_pretrained(model_id)
data_collator = DataCollatorForSeq2Seq(tokenizer)

In [ ]:
def prepare_dataset(sample):
    output = tokenizer(sample["Contents"])
    output["labels"] = tokenizer(sample["IPA"])['input_ids']
    output["length"] = len(output["labels"])
    #print(output)
    return output


ds_train = ds_train.map(prepare_dataset, remove_columns=ds_train.column_names)
ds_eval = ds_eval.map(prepare_dataset, remove_columns=ds_eval.column_names)

In [ ]:
print(ds_train)

In [ ]:
import numpy as np
from datasets import load_metric

wer_metric = load_metric("wer")


def compute_metrics(eval_preds):
    preds, labels = eval_preds
    
    if isinstance(preds, tuple):
        preds = preds[0]
    
    preds = np.where(preds != -100, preds, tokenizer.pad_token_id)
    decoded_preds = tokenizer.batch_decode(preds, skip_special_tokens=True)

    labels = np.where(labels != -100, labels, tokenizer.pad_token_id)
    decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)

    result = wer_metric.compute(predictions=decoded_preds, references=decoded_labels)
    return {"wer": result}

In [ ]:
from transformers import Seq2SeqTrainer, Seq2SeqTrainingArguments

model_id = "bytet5-banglaregional-text-to-ipa-Final"
training_args = Seq2SeqTrainingArguments(
    output_dir=model_id,
    group_by_length=True,
    length_column_name="length",
    per_device_train_batch_size=4,
    per_device_eval_batch_size=4,
    gradient_accumulation_steps=4,
    evaluation_strategy="steps",
    metric_for_best_model="wer",
    greater_is_better=False,
    load_best_model_at_end=False,
    num_train_epochs=10,
    max_steps=18000,
    save_steps=1000,
    eval_steps=1000,
    logging_steps=1000,
    learning_rate=3e-4,
    weight_decay=1e-2,
    warmup_steps=1000,
    save_total_limit=2,
    predict_with_generate=True,
    generation_max_length=512,
    push_to_hub=False,
    report_to="none",
)

In [ ]:
trainer = Seq2SeqTrainer(
    model=model,
    tokenizer=tokenizer,
    args=training_args,
    train_dataset=ds_train,
    eval_dataset=ds_eval,
    data_collator=data_collator,
    compute_metrics=compute_metrics
)

trainer.train()

In [ ]:
trainer.save_model(model_id)

## Inference 

In [ ]:
import pandas as pd
from transformers import pipeline

In [ ]:
test_df = pd.read_csv("/kaggle/input/regipa/test_regipa.csv", index_col="Index")
test_df = test_df.drop(columns=['District'])

In [ ]:
pipe = pipeline("text2text-generation", model="/kaggle/working/bytet5-banglaregional-text-to-ipa-Final/checkpoint-17000/", device=0)

In [ ]:
# preprocess-1: sort by length
index = test_df["Contents"].str.len().sort_values(ascending=False).index
test_df = test_df.reindex(index)

In [ ]:
# preprocess-2: removing english alphabets
alpha_pat = "[a-zA-Z]"
test_df["Contents"] = test_df["Contents"].str.replace(alpha_pat, "", regex=True)

In [ ]:
# preprocess-3: adding period at the end of those texts not having period or '।'
indices_to_add_period = test_df[~test_df['Contents'].str.endswith('।')].index
test_df.loc[indices_to_add_period, 'Contents'] += '।'

In [ ]:
%%time
texts = test_df["Contents"].tolist()
ipas = pipe(texts, max_length=512, batch_size=64)
ipas = [ipa["generated_text"] for ipa in ipas]

In [ ]:
# postprocess-1
test_df["ipa"] = ipas
test_df = test_df.sort_index()

In [ ]:
# postprocess-2: removing period from the end of those ipas in which texts period were added in preprocess-3
test_df.loc[indices_to_add_period, 'ipa'] = test_df.loc[indices_to_add_period, 'ipa'].str.rstrip('।')

In [ ]:
test_df.index.names = ['id']
test_df = test_df.drop(columns=['Contents'])
test_df.rename(columns={'ipa': 'string'},inplace=True)
test_df.head()

In [ ]:
test_df.to_csv("submission_bytebase.csv")